In [2]:
from abc import ABC, abstractmethod
from datetime import datetime, date, timedelta

class User():
    def __init__(self, Id, Name, Email, Password, Users_table_name, Health_table_names, Goal_table_name):
        self.__Id = Id
        self.__Name = Name
        self.__Email = Email
        self.__Password = Password
        self.__Database = Database
        self.__Users_table_name = Users_table_name
        self.__Diet_table_name = Health_table_names[0]
        self.__Fitness_table_name = Health_table_names[1]
        self.__Sleep_table_name = Health_table_names[2]
        self.__Goal_tabl_name = Goal_table_name
        self.__Diet = Diet(self.__Diet_table_name, self.__Goal_table_name, self.__Database, self.__Id)
        self.__Fitness = Fitness(self.__Diet_table_name, self.__Goal_table_name, self.__Database, self.__Id)
        self.__Sleep = Sleep(self.__Diet_table_name, self.__Goal_table_name, self.__Database, self.__Id)
        self.__insert_in_database()
    
    def __insert_in_database(self):
        data_dict['Id'] = self.__Id
        data_dict['Name'] = self.__Name
        data_dict['Email'] = self.__Email
        data_dict['Password'] = self.__Password
        
        try:
            self.__database.insert_row(self.__Users_table_name,data_dict)
            return True
        except ValueError:
            print(f'Could not make an entry in {self.__Users_table_name}')
    
    def get_Diet(self):
        return self.__Diet
    
    def get_Fitness(self):
        return self.__Fitness
    
    def get_Sleep(self):
        return self.__Sleep

class Health(ABC):
 
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id
        super().__init__()
    
    # Get the value corresponding to a particular column for a particular date
    # So for diet it could be the value corresonding to a particular nutrient column
    # For fitness it could the Minutes or the CaloriesBurned column
    # While for sleep it will just be the Minutes (the sleep duration)
    @abstractmethod
    def get_value_given_date(self):
        pass
    
    # Get the daily nutrient/exercise? decomposition
    # For diet it will be the nutrient decomposition for that day.
    # For fitness it will be the workout decomposition for that based on all the exercises 
    # done in that day.
    @abstractmethod
    def get_daily_decomposition(self):
        pass

    # Get the values corresponding to a particular column given a range of dates
    # Same as get_value_given_date, its just for a range of dates.
    @abstractmethod
    def get_values_range(self):
        pass
    
    # Get the daily values.
    # For diet it could be the value of a particular nutrient or the calories?
    # For fitness it could be the Minutes of active time or the CaloriesBurned
    # For sleep it will be the duration of sleep
    @abstractmethod
    def get_daily_value(self):
        pass
    
    # Call get_values_range with todays date and duration = 7 i.e. range (today-7, today)
    @abstractmethod
    def get_weekly_values(self):
        pass
    
    # Call get_values_range with todays date and duration = 28 i.e. range (today-28, today)
    @abstractmethod
    def get_monthly_values(self):
        pass
    
    # API calling function. So for diet it will present the 5 options to the user to select from.
    # Similarly for fitness .....
    @abstractmethod
    def get_from_API(self):
        pass
    
    # Insert the user data into the database after checking for the correctness of the inputs received
    # from the frontend.
    @abstractmethod
    def insert_in_database(self):
        pass
    
    # Inserts the user goals data into the goals database.
    # Common function across all the three features as the goals table is also common.
    # Goals table will have columns: UserID, Type (corresponding to D, F, S) and Value (Float) which
    # corresponds to Calories for diet, Active time for fitness and a
    
    def set_goals(self, goal_dict):
        # Description: 
        # Makes a new entry in the goal table. We will receive Type and the Value from the goal_dict
        # and the Date and UserID will be manually added in this file.
        # 
        # Input:
        # Dictionary with keys: Type (which corresonds to D, F, S), and the Value
        # So for diet we will have calories, fitness we have active time and for sleep we have duration.
        #
        # Output:
        # Returns True if we set the goal correctly otherwise False
        
        input_key_list = ['Type', 'Value']
        ct = 0
        for k in goal_dict.keys():
            if k not in input_key_list:
                ct += 1
        assert ct == 0
        
        # Add Date and UserID to the dictionary before saving it in the database.
        goal_dict['Datetime'] = datetime.now()
        goal_dict['UserID'] = self.__user_id
        
        try:
            self.__database.insert_row(self.__goal_table_name, goal_dict)
            return True
        except:
            return False 
        
    def get_goals(self, Type):
    
    # Description:
    # Returns the latest goal of Type Type from the goal database.
    #
    # Input:
    # Type (CHAR): D,F,S
    #
    # Output:
    # Dictionary with key: Type and the corresponding value.
        
        query = f"SELECT Value FROM {self.__goal_table_name}\
        join Users on Users.id={self.__goal_table_name}.UserID\
        WHERE Type = {Type} ORDER BY Date DESC LIMIT 1"

        try:
            records = self.__database.select_date(query)
            goal_out_dict = {Type: records[0][0]}
            return goal_out_dict
        except ValueError:
            print(f"{Type} Goal not yet set")
        
#     @abstractmethod
#     def check_goals(self):
#         pass

class Diet(Health):
    
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id
    
    def get_value_given_date(self, nutrient, given_date):
        
        # Description: 
        # Takes a nutrient as input for instance protein and a date and return the
        # total amount proteins consumed on that date.
        
        # Input:
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        # given_date(DATE): python date, not datetime
        
        # Output:
        # Float value
        
        next_date = given_date + timedelta(days=1)
        
        query = f"SELECT ServingSize, {nutrient} FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE Datetime >= '{str(given_date)} 00:00:00' AND\
                Datetime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            # If you can find the corresponding date in the database return the total 
            # nutrient consumed, otherwise just throw error.
            records = self.__database.select_data(query)
            given_date_nutrients = 0
            for r in records:
                given_date_nutrients += r[0] * r[1]
            return given_date_nutrients
        except ValueError:
            print(f"No entry for {given_date}.")
            # TODO
            # OR SHOULD WE RAISE AN ERROR INSTEAD OF RETURNING 0?
#             return 0
    
    def get_daily_value(self, nutrient):
        
        # Description:
        # Useful for the main Diet page where we show total calories consumed.
        # For calories we will just call: get_daily_value('cal')
        
        # Input: 
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        
        # Output: 
        # Float value
        
        todays_date = date.today()
        try:
            return self.get_value_given_date(nutrient, todays_date)
        except ValueError:
            print('No entry has been for today')
    
    def get_top2_foods(self, nutrient):
        
        # Description:
        # Return the top 2 food items with the higest quantity of a particular nutrient for that date.
        # Returns a dictionary with keys: the names of the two items and the values = nutrient quantity.
        #
        # Input: 
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        #
        # Output:
        # top2_dict (Dictionary): keys: names of the top two items
        # values: the corresponding nutrient quantity.
        
        todays_date = date.today()
        next_date = given_date + timedelta(days=1)
        
        query = f"SELECT Item, ServingSize, {nutrient} FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE Datetime >= '{str(given_date)} 00:00:00' AND\
                Datetime <= '{str(next_date)} 00:00:00'"
        
        top2_dict = {}
        
        try:
            records = self.__database.select_data(query)
            item_list = []
            nutri_list = []
            for r in records:
                item_list.append(r[0])
                nutri_list.append(r[1] * r[2])
            
            import numpy as np
            nutri_list = -np.array(nutri_list)
            sorted_loc = np.argsort(nutri_list)
            
            k = min(2, len(records))
            
            for i in range(k):
                top2_dict[item_list[i]] = nutri_list[i]
            
            return top2_dict
        except ValueError:
            print('Top 2 items not found.')
            # TODO
            # OR SHOULD WE RAISE AN ERROR INSTEAD OF RETURNING 0?
#             return 0
    
    def get_daily_decomposition(self, nutrient_list): 
        
        # Description:
        # Useful for the diet page where we show the daily percentage breakup.
        # Look 2 diagram in Figure 1 in report.
        
        # Input:
        # nutrient_list (LIST of STRING): ['Cals, Protein, Fat, Carbs, Fiber']
        # There wont be cals because I think cals is a measure of energy and not a measure
        # of the mass, so we only have 4.
        
        # Output:
        # Returns a nested dictionary:
        # First level keys are the nutrients: 'Cals, Protein, Fat, Carbs, Fiber'
        # Second level keys: for each nutrient we have two keys: percentage corresponding to the daily %
        # and top_2 which is another dictionary with keys as the names of the top2 food items and the values
        # corresponding to the quantity of the nutrient.
        
        try:
            nutri_decom = {}
            for nutri in nutrient_list:
                nutri_decom[nutri] = self.get_daily_value(nutri)

            total = sum([nutri_decom[k] for k in nutri_decom.keys()])

            per_nutri_decom = {}
            for nutri in nutrient_list:
                per_nutri_decom[nutri] = {}
                per_nutri_decom[nutri]['percentage'] = 100.0 * (nutri_decom[nutri]/total)
                per_nutri_decom[nutri]['top_2'] = self.get_top2_foods(nutri)

            return per_nutri_decom
        except ValueError:
            print('No entry has been made for today.')
    
    def get_values_range(self, nutrient, date, duration):
        
        # Description: 
        # Useful for making the weekly and monthly (if we are doing it) charts.
        # Just need to pass the start date and the duration, and it computes the 
        # values corresponding to a particular nutrient for that duration.
        
        # Input:
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        # date (DATE): python date, not datetime
        # duration (INT): the number of days before the given date. So for instance
        # for weekly date duration will be 7 and we will compute (date-7, date)
        
        # Output:
        # Returns a dictionary with the dates of that duration (date-duration, date) as the keys and the 
        # corresponding nutrient value as the values of the dictionary.
        
        nutrients_dict = {}
        for i in range(duration):
            dt = date - timedelta(days=i)
            try:
                nutrients_dict[str(dt)] = self.get_value_given_date(nutrient, dt)
            except:
                if nutrients_dict:
                    return nutrients_dict
                else:
                    raise ValueError('No entry has been made yet.')
        return nutrients_dict
    
    def get_weekly_values(self, nutrient):
        
        # Description:
        # Weekly values for a particular nutrient.
        # Just call the previous function with duration = 7
        
        # Input:
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last week and the corresponding 
        # nutrient values.
        try:
            return self.get_values_range(nutrient, date.today(), 7)
        except ValueError:
            print('Week has not yet been completed.')
    
    def get_monthly_values(self, nutrient):
        
        # Description:
        # Sames as before just for a month.
        
        # Input:
        # nutrient (STRING): 'Cals, Protein, Fat, Carbs, Fiber'
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last month and the corresponding 
        # nutrient values.
        
        try:
            return self.get_values_range(nutrient, date.today(), 28)
        except ValueError:
            print('Month has not yet been completed.')        
            
    def get_from_API(self, query, upc):
        
        # Description: 
        # Calculates the nutrient values from the API
        
        # Input:
        # query (STRING): user input i.e. the string which user enters.
        # upc (BOOL): True if the query is a barcode number and False otherwise
        
        # Output:
        # api_dict (Dictionary): Nested Dictionary with keys as the nutrient names: 
        # First level keys: 0,1,2,3,4 corresponding to the top 5 choices we return
        # Second level keys: Label and Nutrients. Labels is the name for the ith item
        # and Nutrients is another dictionary with keys: 'Cals, Protein, Fat, Carbs, Fiber'
        #
        # See edamam.py for more details.
        #
        # success (BOOL): return True if an item with the query name is found in the API otherwise returns False.
        
        food_options_dict, success = self.get_nutrient_information(query, upc)
        if success:
            return label, api_dict, success
        else:
            raise ValueError('Could not find the item in the FoodAPI.')
    
    
    # Since we are assuming that we will be providing the user with options to select from, we will let the
    # the input to this function be the following nested dictionary:
    # First level keys: Item (corresponding to the item name which will be either what the api returns
    # or if the user does not choose api then the manual entry), ServingSize (the serving size), Barcode (a 
    # boolean which will be true if the user chooses barcode) and finally nutri_dict (which is the nutrient
    # dictionary with keys: 'Cals, Protein, Fat, Carbs, Fiber' and their corresponding values.)

    # Input dict should have the following keys:
    # 1)Item
    # 2)ServingSize
    # 3)Barcode
    # 4)nutri_dict
    
    def insert_in_database(self, input_dict):
        
        # Description:
        # Inserts the input from the front end in the table.
        # 
        # Input:
        # input_dict (Dictionary): input dictionary from the user, described above
        #
        # Output:
        # Returns True if the entry is correctly made
        # Else returns False
        
        nutri_name_list = ['Cals', 'Protein', 'Fat', 'Carbs', 'Fiber']
        input_key_list = ['Item', 'ServingSize', 'Barcode', 'nutri_dict']
    
        ct = 0
        for k in input_dict.keys():
            if k not in input_key_list():
                ct+=1
        if ct == 0:
            for k in input_dict['nutri_dict']:
                if i not in nutri_name_list:
                    ct+=1
        assert ct == 0
        
        data_dict = input_dict
        del data_dict['nutri_dict']
        date_dict['UserID'] = self.__user_id
        data_dict['Datetime'] = datetime.now()
        
        for k in input_dict['nutri_dict'].keys():
            data_dict[k] = input_dict['nutri_dict'][k]

        try:
            self.__database.insert_row(self.__table_name,data_dict)
            return True
        except ValueError:
            print(f'Could not make an entry in {self.__table_name}')

#     def check_goal(self, nutrient):
#         todays_date = date.today()
#         todays_value = self.get_value_date(nutrient, todays_date)
#         c = self.__database.get_Cursor()
#         query = f"SELECT {nutrient} FROM {self.__goal_table_name}\
#                 ORDER BY ID DESC LIMIT 1"

#         try:
#             c.execute(query)
#             records = c.fetchall()
#             goal_value = records[0]
#             if todays_value >= goal_value:
#                 return False
#             else:
#                 return True
#         except:
#             False

class Fitness(Health):
    
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id
        
    def get_value_given_date(self, min_or_calburn, given_date):
        
        # Description: 
        # Takes exercise/workout type as input for instance running and a date and returns the
        # total active time for that workout type.
        
        # Input:
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        # given_date(DATE): python date, not datetime
        
        # Output:
        # Float value
        
        next_date = given_date + timedelta(days=1)
        
        query = f"SELECT {min_or_calburn} FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE Datetime >= '{str(given_date)} 00:00:00' AND\
                Datetime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            # If you can find the corresponding date in the database return the total 
            # active time for the exercise, otherwise just return 0
            records = self.__database.select_data(query)
            active_time = 0
            for r in records:
                active_time += r[0] 
            return active_time
        except ValueError:
            print(f"No entry for {given_date} {ex_type}")
            # TODO
            # OR SHOULD WE RAISE AN ERROR INSTEAD OF RETURNING 0?
#             return 0
        
#     def get_total_value_given_date(self, min_or_calburn, given_date):
        
#         # Description: 
#         # Takes exercise/workout type as input for instance running and a date and returns the
#         # total active time for that day. So it adds the active time of all the workouts for that day.
        
#         # Input:
#         # min_or_calburn (STRING): Minutes, CaloriesBurned
#         # given_date(DATE): python date, not datetime
        
#         # Output:
#         # Float value
        
#         next_date = given_date + timedelta(days=1)
        
#         query = f"SELECT {min_or_calburn} FROM {self.__table_name}\
#                 join Users on Users.id={self.__table_name}.{self.__user_id}\
#                 WHERE Date >= '{str(given_date)} 00:00:00' AND\
#                 Date <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

#         try:
#             # If you can find the corresponding date in the database return the total 
#             # active time for the exercise, otherwise just return 0
#             records = self.__database.select_data(query)
#             active_time = 0
#             for r in records:
#                 active_time += r 
#             return active_time
#         except ValueError:
#             print(f"No entry for {given_date}")
#             # TODO
#             # OR SHOULD WE RAISE AN ERROR INSTEAD OF RETURNING 0?
# #             return 0
        
    def get_daily_value(self, min_or_calburn):
        
        # Description:
        # Useful for the summary page/main page where we show total active time.
        # Just call the above function with todays date.
        
        # Input: 
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        
        # Output: 
        # Returns the total active time for today.
        
        todays_date = date.today()
        try:
            return self.get_value_given_date(min_or_calburn, todays_date)
        except ValueError:
            print('No entry has been made for today')
            
    
    def get_daily_decomposition(self, min_or_calburn): 
        
        # Description:
        # Useful for the fitness page where we show the daily percentage breakup.
        # So basically we could give the percentage of the exercises the user performed that day.
        # Unlike diet, we dont fix the nutrient list. Instead we scan throught all the exercise
        # done in a particular day and look for the unique entries and just return the total for each
        # in the form of a dictionary.
        
        # Input:
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        
        # Output:
        # Returns a dictionary with keys as the the exercise type and the corresponding active time as the value.
        
        
        todays_date = date.today()
        next_date = todays_date + timedelta(days=1)
        
        query = f"SELECT WorkoutType, {min_or_calburn} FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE Datetime >= '{str(todays_date)} 00:00:00' AND\
                Datetime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            ex_type_list = []
            active_time_list = []
            # If you can find the corresponding date in the database return the total 
            # active time for the exercise, otherwise just return 0
            records = self.__database.select_data(query)
            active_time = 0
            for r in records:
                ex_type_list.append(r[0])
                active_time_list.append(r[1])
                
            ex_type_set = set(ex_type_list)
            ex_decom = {}
            for ex in ex_type_set:
                at = 0
                for  e,a in zip(ex_type_list, active_time_list):
                    if ex == e:
                        at += a
                ex_decom[ex] = at
                    
            total = sum([ex_decom[k] for k in ex_decom.keys()])
            for ex in ex_decom.keys():
                ex_decom[ex]/=total
            return ex_decom

        except ValueError:
            print(f"No entry for today")
            
    def get_daily_workout_list(self):
        # Description: 
        # Returns a dictionary with all the workouts for today. See FrontEnd Outline point 5, c, iii
        # This can then be just displaed with cards.
        # 
        # Input:
        #
        # Output: 
        # Returns a dictionary with first level keys: 0,1,2 corresponding to the number of 
        # workouts done for the day. Second level keys: WorkoutType, Minutes, CaloriesBurned
        # and Time. As mentioned in the FrontEnd Outline diagram.
        
        todays_date = date.today()
        next_date = todays_date + timedelta(days=1)
        
        query = f"SELECT WorkoutType, Minutes, CaloriesBurned, Date, FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE Datetime >= '{str(todays_date)} 00:00:00' AND\
                Datetime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            todays_dict = {}
            records = self.__database.select_data(query)
            for i,r in enumerate(records):
                todays_dict[i] = {}
                todays_dict[i]['WorkoutType'] = r[0]
                todays_dict[i]['Minutes'] = r[1]
                todays_dict[i]['CaloriesBurned'] = r[2]
                todays_dict[i]['Time'] = datetime.strptime(r[3], "%m/%j/%y %H:%M")
                
                return todays_dict
            
        except ValueError:
            print(f"No entry for today")
        
            
    def get_values_range(self, min_or_calburn, date, duration):
        
        # Description: 
        # Useful for making the weekly and monthly (if we are doing it) charts.
        # Just need to pass the start date and the duration, and it computes the 
        # values corresponding to the active time for that duration.
        
        # Input:
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        # date (DATE): python date, not datetime
        # duration (INT): the number of days before the given date. So for instance
        # for weekly date duration will be 7 and we will compute (date-7, date)
        
        # Output:
        # Returns a dictionary with the dates of that duration (date-duration, date) as the keys and the 
        # corresponding active times value as the values of the dictionary.
        
        at_dict = {}
        for i in range(duration):
            dt = date - timedelta(days=i)
            # We use try catch here because when the total number of entried are lesser thatn the
            # duration we just return whatever we have. For instance when there is only 4 days of
            # data (i.e. the first 4 days.) and the user calls the weekly method, 
            # we just return the data for the first 4 days.
            # This will come into play only at the start, when at least one week needs to pass.
            try:
                at_dict[str(dt)] = self.get_value_given_date(min_or_calburn, dt)
            except:
                if at_dict:
                    return at_dict
                else:
                    raise ValueError('No entry has been made yet.')
        return at_dict
    
    
    def get_weekly_values(self, min_or_calburn):
        
        # Description:
        # Weekly values for a active time or calories burned.
        # Just call the previous function with duration = 7
        
        # Input:
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last week and the corresponding 
        # active times or calories burned values.
        try:
            return self.get_values_range(min_or_calburn, date.today(), 7)
        except ValueError:
            print('Week has not yet been completed.')
    
    def get_monthly_values(self, min_or_calburn):
        
        # Description:
        # Sames as before just for a month.
        
        # Input:
        # min_or_calburn (STRING): Minutes, CaloriesBurned
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last month and the corresponding 
        # minutes or calories burned.
        
        try:
            return self.get_values_range(min_or_calburn, date.today(), 28)
        except ValueError:
            print('Month has not yet been completed')    
       
        
    def get_from_API(self):
        # TODO
        return calories_burnt
    
    
    # The input dictionary should having keys: WorkoutType, Minutes. Before storing we compute
    # the calories burned from the api and then store it in the database.
    
    def insert_in_database(self, input_dict):
        
        # Description:
        # Inserts the input from the front end in the table.
        # 
        # Input:
        # input_dict (Dictionary): input dictionary from the user, described above
        #
        # Output:
        # Returns True if the entry is correctly made
        # Else returns False
        
        input_key_list = ['WorkoutType', 'Minutes']
    
        ct = 0
        for k in input_dict.keys():
            if k not in input_key_list():
                ct+=1
        assert ct == 0
        
        data_dict = input_dict
        date_dict['UserID'] = self.__user_id
        data_dict['Date'] = datetime.now()
        data_dict['CaloriesBurnt'] = get_from_API()

        try:
            self.__database.insert_row(self.__table_name,data_dict)
            return True
        except ValueError:
            print(f'Could not make an entry in {self.__table_name}')
    
#     def check_goal(self, key, ex_list):
#         todays_date = date.today()
#         todays_value = self.get_total_value_date(ex_list, todays_date)
#         c = self.__database.get_Cursor()
#         query = f"SELECT {key} FROM {self.__goal_table_name}\
#                 ORDER BY ID DESC LIMIT 1"

#         try:
#             c.execute(query)
#             records = c.fetchall()
#             goal_value = records[0]
#             if todays_value >= goal_value:
#                 return False
#             else:
#                 return True
#         except:
#             False

class Sleep(Health):
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id   
        
    def get_minutes_from_diff_dates(self, d1, d2):
        difference = d2 - d1
        return divmod(difference.days * seconds_in_day + difference.seconds, 60)[0]
    
#     def check_nap(self, d1, d2):
#         # Description:
#         # Checks if the given sleep entry is a nap or not. If the duration is less than 4 hours its a 
#         # nap otherwise it will be regular sleep
#         #
#         # Input:
#         # Two datetimes 
#         #
#         # Output:
#         # returns True if its a nap otherwise false
#         time_min = self.get_minutes_from_diff_dates(d1, d2)
#         if time_min < 240:
#             return True
#         else:
#             return False
        
    def get_value_given_date(self, nap, given_date):
        
        # Description: 
        # Returns the total sleep or nap duration for a particular day.
        
        # Input:
        # nap (BOOL): true if you want nap otherwise false
        # given_date(DATE): python date, not datetime
        
        # Output:
        # Float value
        
        next_date = given_date + timedelta(days=1)
        
        # Note that WakeupTime replaces Date from the previous queries. Because all the sleeps
        # which end on a particular day will contribute to the sleep duration for that day.
        
        query = f"SELECT Minutes FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE WakeupTime >= '{str(given_date)} 00:00:00' AND\
                WakeupTime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}
                AND Nap = {nap}"

        try:
            records = self.__database.select_data(query)
            minutes = 0
            for r in records:
                minutes += r[0] 
            return minutes
        except:
            # THe user may not nap everyday. So for nap we will just return 0, if no entry if found
            # But the user will sleep daily, so we raise an error if that is not found.
            if nap:
                return 0
            else:
                raise ValueError(f"No entry for {given_date}")
            
    def get_total_value_given_date(self, given_date):
        
        # Description: 
        # Returns the total sleep + nap duration for a particular day.
        
        # Input:
        # given_date(DATE): python date, not datetime
        
        # Output:
        # Float value 
        try:
            return self.get_value_given_date(True, given_date) + self.get_value_given_date(False, given_date)
        except ValueError:
            print(f"No entries")
        
    def get_daily_value(self):
        
        # Description:
        # Useful for the summary page/main page where we show total sleep time.
        # Just call the above function with todays date.
        
        # Input: 
        
        # Output: 
        # Returns the total sleep time for today.
        
        todays_date = date.today()
        try:
            return self.get_total_value_given_date(todays_date)
        except ValueError:
            print('No entry has been made for today.')
            
    def get_values_range(self, date, duration):
        
        # Description: 
        # Useful for making the weekly and monthly (if we are doing it) charts.
        # Just need to pass the start date and the duration, and it computes the 
        # values corresponding to the  for that duration.
        
        # Input:
        # date (DATE): python date, not datetime
        # duration (INT): the number of days before the given date. So for instance
        # for weekly date duration will be 7 and we will compute (date-7, date)
        
        # Output:
        # Returns a dictionary with the dates of that duration (date-duration, date) as the keys and the 
        # corresponding total sleep durations as the values of the dictionary.
        
        sleep_dict = {}
        for i in range(duration):
            dt = date - timedelta(days=i)
            # We use try catch here because when the total number of entried are lesser thatn the
            # duration we just return whatever we have. For instance when there is only 4 days of
            # data (i.e. the first 4 days.) and the user calls the weekly method, 
            # we just return the data for the first 4 days.
            # This will come into play only at the start, when at least one week needs to pass.
            try:
                sleep_dict[str(dt)] = self.get_total_value_given_date(dt)
            except:
                if sleep_dict:
                    return sleep_dict
                else:
                    raise ValueError('No entry has been made yet.')
        return sleep_dict
    
    
    def get_weekly_values(self):
        
        # Description:
        # Weekly values for sleep duration.
        # Just call the previous function with duration = 7
        
        # Input:
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last week and the corresponding 
        # active times or calories burned values.
        try:
            return self.get_values_range(date.today(), 7)
        except ValueError:
            print('Week has not yet been completed.')
    
    def get_monthly_values(self, min_or_calburn):
        
        # Description:
        # Sames as before just for a month.
        
        # Input:
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last month and the corresponding 
        # minutes or calories burned.
        
        try:
            return self.get_values_range(date.today(), 28)
        except ValueError:
            print('Month has not yet been completed.') 
            
            
    # INput dict will have keys: SleepTime, WakeupTime
    def insert_in_database(self, input_dict):
        
        # Description:
        # Inserts the input from the front end in the table.
        # 
        # Input:
        # input_dict (Dictionary): input dictionary from the user, described above
        #
        # Output:
        # Returns True if the entry is correctly made
        # Else returns False
        
        input_key_list = ['SleepTime', 'WakeupTime']
    
        ct = 0
        for k in input_dict.keys():
            if k not in input_key_list():
                ct+=1
        assert ct == 0
        
        data_dict = input_dict
        date_dict['UserID'] = self.__user_id
        data_dict['Date'] = datetime.now()
        data_dict['Minutes'] = get_minutes_from_diff_dates(datetime.now(), datetime.now()+timedelta(days=1))

        try:
            self.__database.insert_row(self.__table_name,data_dict)
            return True
        except ValueError:
            print(f'Could not make an entry in {self.__table_name}')
        
        
    def get_daily_sleep_list(self):
        # Description: 
        # Returns a dictionary with all the workouts for today. See FrontEnd Outline point 5, c, iii
        #
        # Input:
        #
        # Output: 
        # Returns a dictionary with first level keys: 0,1,2 corresponding to the number of 
        # workouts done for the day. Second level keys: WorkoutType, Minutes, CaloriesBurned
        # and Time. As mentioned in the FrontEnd Outline diagram.
        
        todays_date = date.today()
        next_date = todays_date + timedelta(days=1)
        
        query = f"SELECT SleepTime, WakeupTime, Minutes, FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE WakeupTime >= '{str(todays_date)} 00:00:00' AND\
                WakeupTime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            todays_dict = {}
            records = self.__database.select_data(query)
            for i,r in enumerate(records):
                todays_dict[i] = {}
                todays_dict[i]['WakeupTime'] = r[0]
                todays_dict[i]['SleepTime'] = r[1]
                todays_dict[i]['Minutes'] = r[2]
                
                return todays_dict
            
        except ValueError:
            print(f"No entry for today")
            
    # TODO
    def get_average_times(self, sleep_or_wake):
        # Description: 
        # Returns a dictionary with all the workouts for today. See FrontEnd Outline point 5, c, iii
        #
        # Input:
        #
        # Output: 
        # Returns a dictionary with first level keys: 0,1,2 corresponding to the number of 
        # workouts done for the day. Second level keys: WorkoutType, Minutes, CaloriesBurned
        # and Time. As mentioned in the FrontEnd Outline diagram.
        
        avg_dict = 0
        for i in range(duration):
            dt = date - timedelta(days=i)
            next_dt = dt + timedelta(days=1)
            
            query = f"SELECT {sleep_or_wakeup} Minutes, FROM {self.__table_name}\
            join Users on Users.id={self.__table_name}.UserID\
            WHERE WakeupTime >= '{str(dt)} 00:00:00' AND\
            WakeupTime <= '{str(next_dt)} 00:00:00' AND UserID = {self.__user_id}"
            
            try:
                
                at_dict[str(dt)] = self.get_value_given_date(dt)
            except:
                if dt_dict:
                    return at_dict
                else:
                    raise ValueError('No entry has been made yet.')
        
        todays_date = date.today()
        next_date = todays_date + timedelta(days=1)
        
        query = f"SELECT SleepTime, WakeupTime, Minutes, FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.UserID\
                WHERE WakeupTime >= '{str(todays_date)} 00:00:00' AND\
                WakeupTime <= '{str(next_date)} 00:00:00' AND UserID = {self.__user_id}"

        try:
            todays_dict = {}
            records = self.__database.select_data(query)
            for i,r in enumerate(records):
                todays_dict[i] = {}
                todays_dict[i]['WakeupTime'] = r[0]
                todays_dict[i]['SleepTime'] = r[1]
                todays_dict[i]['Minutes'] = r[2]
                
                return todays_dict 
            
        except ValueError:
            print(f"No entry for today")
            
    